# Part 1

--- Day 12: Rain Risk ---

Your ferry made decent progress toward the island, but the storm came in faster than anyone expected. The ferry needs to take evasive actions!

Unfortunately, the ship's navigation computer seems to be malfunctioning; rather than giving a route directly to safety, it produced extremely circuitous instructions. When the captain uses the PA system to ask if anyone can help, you quickly volunteer.

The navigation instructions (your puzzle input) consists of a sequence of single-character actions paired with integer input values. After staring at them for a few minutes, you work out what they probably mean:

- Action `N` means to move north by the given value.
- Action `S` means to move south by the given value.
- Action `E` means to move east by the given value.
- Action `W` means to move west by the given value.
- Action `L` means to turn left the given number of degrees.
- Action `R` means to turn right the given number of degrees.
- Action `F` means to move forward by the given value in the direction the ship is currently facing.

The ship starts by facing east. Only the `L` and `R` actions change the direction the ship is facing. (That is, if the ship is facing east and the next instruction is `N10`, the ship would move north 10 units, but would still move east if the following action were `F`.)

For example:

```
F10
N3
F7
R90
F11
```

These instructions would be handled as follows:

- `F10` would move the ship 10 units east (because the ship starts by facing east) to east 10, north 0.
- `N3` would move the ship 3 units north to east 10, north 3.
- `F7` would move the ship another 7 units east (because the ship is still facing east) to east 17, north 3.
- `R90` would cause the ship to turn right by 90 degrees and face south; it remains at east 17, north 3.
- `F11` would move the ship 11 units south to east 17, south 8.

At the end of these instructions, the ship's Manhattan distance (sum of the absolute values of its east/west position and its north/south position) from its starting position is `17 + 8 = 25`.

Figure out where the navigation instructions lead. What is the Manhattan distance between that location and the ship's starting position?

In [14]:
(import [dataclasses [dataclass field]])
(import [pathlib [Path]])

In [2]:
(setv INPUT_FILE (/ (.cwd Path) "inputs" "day12" "part1.txt"))

In [3]:
(defn input-lines []
  (with [fp (.open INPUT_FILE)]
    (yield-from (gfor line fp (.strip line)))))

In [23]:
(with-decorator dataclass
  (defclass Instruction []
    (^str direction)
    (^int value)))

In [26]:
(defn input-instructions []
  (yield-from (gfor line (input-lines) (Instruction (get line 0) (int (cut line 1))))))

In [18]:
(with-decorator dataclass
  (defclass Ship []
    (setv ^str facing (field :default "E" :init False))
    (setv ^int north-south (field :default 0 :init False))
    (setv ^int east-west (field :default 0 :init False))))

In [30]:
(defn move-ship-direction [ship direction value]
  ; north +, east +
  (cond
    [(= direction "N") (+= ship.north-south value)]
    [(= direction "S") (-= ship.north-south value)]
    [(= direction "E") (+= ship.east-west value)]
    [(= direction "W") (-= ship.east-west value)]))

In [37]:
(defn turn-ship [ship direction value]
  ; N 0, E 90, S 180 W 270
  (setv current-facing (get {"N" 0 "E" 90 "S" 180 "W" 270} ship.facing))
  ; R +, L -
  (setv new-facing
    (if (= direction "R")
      (% (+ current-facing value) 360)
      (% (- current-facing value) 360)))
  (setv ship.facing (get {0 "N" 90 "E" 180 "S" 270 "W"} new-facing)))

In [46]:
(defn move-ship [ship instr]
  ; north +, east +
  (cond
    [(in instr.direction "NESW") (move-ship-direction ship instr.direction instr.value)]
    [(= instr.direction "F") (move-ship-direction ship ship.facing instr.value)]
    [(in instr.direction "RL") (turn-ship ship instr.direction instr.value)]))

In [44]:
(defn apply-instructions [ship instructions]
  (list (map (fn [instr] (move-ship ship instr)) instructions))
  ship)

In [47]:
(apply-instructions (Ship) (input-instructions))

Ship(facing='S', north_south=-147, east_west=791)

In [48]:
(+ 147 791)

938

# Part 2

Before you can give the destination to the captain, you realize that the actual action meanings were printed on the back of the instructions the whole time.

Almost all of the actions indicate how to move a waypoint which is relative to the ship's position:

- Action `N` means to move the waypoint north by the given value.
- Action `S` means to move the waypoint south by the given value.
- Action `E` means to move the waypoint east by the given value.
- Action `W` means to move the waypoint west by the given value.
- Action `L` means to rotate the waypoint around the ship left (counter-clockwise) the given number of degrees.
- Action `R` means to rotate the waypoint around the ship right (clockwise) the given number of degrees.
- Action `F` means to move forward to the waypoint a number of times equal to the given value.

The waypoint starts 10 units east and 1 unit north relative to the ship. The waypoint is relative to the ship; that is, if the ship moves, the waypoint moves with it.

For example, using the same instructions as above:

- `F10` moves the ship to the waypoint 10 times (a total of 100 units east and 10 units north), leaving the ship at east 100, north 10. The waypoint stays 10 units east and 1 unit north of the ship.
- `N3` moves the waypoint 3 units north to 10 units east and 4 units north of the ship. The ship remains at east 100, north 10.
- `F7` moves the ship to the waypoint 7 times (a total of 70 units east and 28 units north), leaving the ship at east 170, north 38. The waypoint stays 10 units east and 4 units north of the ship.
- `R90` rotates the waypoint around the ship clockwise 90 degrees, moving it to 4 units east and 10 units south of the ship. The ship remains at east 170, north 38.
- `F11` moves the ship to the waypoint 11 times (a total of 44 units east and 110 units south), leaving the ship at east 214, south 72. The waypoint stays 4 units east and 10 units south of the ship.

After these operations, the ship's Manhattan distance from its starting position is `214 + 72 = 286`.

Figure out where the navigation instructions actually lead. What is the Manhattan distance between that location and the ship's starting position?

In [51]:
(with-decorator dataclass
  (defclass Point []
    (^int north-south)
    (^int east-west)))

In [53]:
(defn move-point-direction [point direction value]
  ; north +, east +
  (cond
    [(= direction "N") (+= point.north-south value)]
    [(= direction "S") (-= point.north-south value)]
    [(= direction "E") (+= point.east-west value)]
    [(= direction "W") (-= point.east-west value)]))

In [55]:
(defn rotate-point [ship point direction value]
  ; rotate point around ship the given direction by the angle given in value
  (setv north-south (- point.north-south ship.north-south))
  (setv east-west (- point.east-west ship.east-west))
  ; standardize on clockwise rotation
  (setv rotation
    (if (= direction "R")
      value
      (- 360 value)))
  (cond
    [(= rotation 90)
     (do
       (setv point.north-south (- ship.north-south east-west))
       (setv point.east-west (+ ship.east-west north-south)))]
    [(= rotation 180)
     (do
       (setv point.north-south (- ship.north-south north-south))
       (setv point.east-west (- ship.east-west east-west)))]
    [(= rotation 270)
     (do
       (setv point.north-south (+ ship.north-south east-west))
       (setv point.east-west (- ship.east-west north-south)))]))

In [62]:
(do (rotate-point (Point 0 0) (setx temp (Point 1 10)) "L" 180) temp)

Point(north_south=-1, east_west=-10)

In [64]:
(defn ship-toward-point [ship point times]
  (setv north-south (- point.north-south ship.north-south))
  (setv east-west (- point.east-west ship.east-west))
  (+= ship.north-south (* north-south times))
  (+= point.north-south (* north-south times))
  (+= ship.east-west (* east-west times))
  (+= point.east-west (* east-west times)))

In [65]:
(defn do-one-instruction [ship point instr]
  (cond
    [(in instr.direction "NESW") (move-point-direction point instr.direction instr.value)]
    [(in instr.direction "RL") (rotate-point ship point instr.direction instr.value)]
    [(= instr.direction "F") (ship-toward-point ship point instr.value)]))

In [66]:
(defn apply-instructions2 [ship point instructions]
  (for [instr instructions] (do-one-instruction ship point instr)))

In [68]:
(do
  (apply-instructions2
    (setx ship (Point 0 0))
    (setx point (Point 1 10))
    (input-instructions))
  (, ship point))

(Point(north_south=-29011, east_west=-25393), Point(north_south=-29000, east_west=-25504))

In [69]:
(+ 29011 25393)

54404